# Load libraries

In [1]:
import tensorflow.compat.v1 as tf
from tensorflow.compat.v1.keras.models import load_model
import tensorflow.compat.v1.keras.backend as K
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.colors import ListedColormap

In [3]:
import cv2
import glob
import time
import gc

# Functions

In [4]:
def get_iterate(model, layer):
    # Get the output of the layer and the data
    data_output = model.output[:, 0]

    # Get layer output
    get_layer = model.get_layer(layer)
    layer_output = get_layer.output

    # Calculate the gradients
    grads = K.gradients(data_output, layer_output)[0]

    pooled_grads = K.mean(grads, axis = (0, 1))

    iterate = K.function([model.input], [pooled_grads, layer_output[0]])
    
    return(iterate)

In [5]:
def get_heatmap_matrix(dataset, nth_data, iterate):

    # Get the data
    data = dataset[nth_data:nth_data+1, :, :]
    
    pooled_grads_value, layer_output_value = iterate([data])

    layer_output_value[:, ] *= pooled_grads_value
        
    # Get the heatmap matrix
    heatmap = np.copy(layer_output_value)
    heatmap = np.mean(heatmap, axis = -1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    # Expand the dimensionality of heatmap so that it can be plot
    heatmap = np.expand_dims(heatmap, axis=0)
    
    return heatmap

In [6]:
def load_npy_data(filename):
    # Define pattern of the npy data
    data_loading_pattern = "data-sets/" + filename + ".npy"
    data_loading_path = glob.glob(data_loading_pattern)

    # Load the data
    print("Loading data:", str(filename), "from " + data_loading_path[0])
    data = np.load(data_loading_path[0])
    
    return(data)

In [7]:
def get_sum_heatmap(data_name, model, layer, from_sample, sample_length):
    
    data = load_npy_data(data_name)
    print("Data shape: " + str(data.shape[0]))
    
    iterate = get_iterate(model, layer)
    result = []

    min_sample = from_sample
    max_sample = np.min([from_sample + sample_length, data.shape[0]])
    
    print("")
    # For loop to get the heatmap for each matrix
    for nth_sample in range(min_sample, max_sample):
        print("Getting the heatmap for the data: sample " + str(nth_sample) + "/" + str(max_sample-1))
        heatmap = get_heatmap_matrix(data, nth_sample, iterate)

        # Put all of the reults together
        result.append(heatmap)
        
        # Free memory (useless)
        gc.collect()

    # Change the list to numpy array
    all_matrices = np.array(result)

    # Save all matrices
    saving_path = "results/all_matrices_" + data_name + "_" + layer + "_samples_" + str(min_sample).zfill(4) + "_" + str(max_sample-1).zfill(4) + ".npy"

    print("")
    print("Saving results in", saving_path)
    np.save(saving_path, all_matrices)

# Run functions in steps

In [8]:
model = load_model("models/weights/cnn_lstm_30-0.41.hdf5")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [9]:
get_sum_heatmap(
    data_name ="x_train", 
    model = model,  
    layer = "conv1d_4",
    from_sample = 425,
    sample_length = 25
)

Loading data: x_train from data-sets/x_train.npy
Data shape: 462

Getting the heatmap for the data: sample 425/449
Getting the heatmap for the data: sample 426/449
Getting the heatmap for the data: sample 427/449
Getting the heatmap for the data: sample 428/449
Getting the heatmap for the data: sample 429/449
Getting the heatmap for the data: sample 430/449
Getting the heatmap for the data: sample 431/449
Getting the heatmap for the data: sample 432/449
Getting the heatmap for the data: sample 433/449
Getting the heatmap for the data: sample 434/449
Getting the heatmap for the data: sample 435/449
Getting the heatmap for the data: sample 436/449
Getting the heatmap for the data: sample 437/449
Getting the heatmap for the data: sample 438/449
Getting the heatmap for the data: sample 439/449
Getting the heatmap for the data: sample 440/449
Getting the heatmap for the data: sample 441/449
Getting the heatmap for the data: sample 442/449
Getting the heatmap for the data: sample 443/449
Get